In [284]:
import numpy as np
from PIL import Image
import glob

IMG_SHAPE = (125, 125)

def crop_image(img):
    width, height = img.size

    new_width = IMG_SHAPE[0]
    new_height = IMG_SHAPE[1]
    left = (width - new_width)/2 - .5
    top = (height - new_height)/2 - .5
    right = (width + new_width)/2 - .5
    bottom = (height + new_height)/2 - .5

    # Crop the center of the image
    im = img.crop((left, top, right, bottom))

    return im

def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')
    img = crop_image(img)

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    print(flat_array)

    return flat_array


def vector_to_img(vector, shape = IMG_SHAPE):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img


def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [285]:
all_imgs = []

# Get images from dataset and convert them to vectors
for folder in glob.iglob('../dataset/*'):
    for img in glob.iglob(folder + '/*'):
        all_imgs.append(img_to_vector(img))

# Create an np.array from the vectors
training_set = np.array(all_imgs)


[ 40  37  36 ... 189 189 189]
[188 198 207 ... 145 140 135]
[102 101 100 ...  66  62  57]
[194 182 175 ... 128 128 126]
[50 49 69 ... 55 76 84]
[172 173 170 ...  87  85  82]
[184 185 185 ... 176 175 172]
[221 220 219 ...  74  71  79]
[34 34 34 ... 53 62 76]
[223 224 227 ...  93  94  99]
[ 72  90 111 ... 153 152 152]
[184 182 179 ...  58  59  62]
[51 83 99 ... 23 22 28]
[177 177 177 ... 177 169 157]
[168 175 182 ...  99 118 140]
[64 56 44 ... 50 50 50]
[ 70  70  69 ... 164 182 170]
[ 94  99 102 ...  95  93  90]
[65 80 81 ... 38 39 41]
[ 26  26  28 ... 135 118  96]
[ 24  26  32 ... 228 228 228]
[ 34  29  33 ... 188 188 188]
[254 255 247 ...  73  86 100]
[124  90  97 ...  62  59  56]
[ 97 100  99 ... 207 209 172]
[232 232 232 ... 242 242 242]
[121 123 137 ... 122 122 121]
[175 174 172 ...  52  47  45]
[29 29 29 ... 12 11  9]
[ 56  82 116 ... 166 166 166]
[242 242 244 ... 246 245 244]
[72 74 75 ... 47 47 47]
[234 236 239 ...  28  25  22]
[ 47  46  47 ... 129 141 154]
[130 125 117 ...  27  

In [286]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [287]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
sub = None
for v in training_set:
    sub = np.subtract(v, avg_face)
    A.append(sub)

print(np.array(A).shape)

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = np.array(A).transpose()

(225, 15625)


In [288]:
# Form the covariance matrix
cov_matrix = np.cov(np.array(A))

In [289]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
eigvectors_sort

[array([ 3.64590622e-02, -7.04102524e-02,  2.11604431e-03, -7.54615526e-02,
        -3.30164833e-02,  5.32767515e-02, -1.15014794e-02,  1.49721093e-03,
         1.36225111e-01, -1.99853853e-02, -6.19208830e-02,  2.00325986e-03,
        -9.61887369e-02, -6.84612745e-02, -3.58007691e-02,  2.57351506e-02,
        -4.14546721e-02, -6.71044383e-02,  5.62005186e-02,  2.20441298e-02,
        -2.02574015e-01, -9.58062314e-02, -8.73126378e-02, -1.94416432e-02,
        -6.84615390e-02, -8.95677149e-02, -1.04697575e-01, -1.20969854e-01,
         1.64264372e-01, -7.08937451e-02, -1.18894974e-01, -5.89083531e-03,
        -5.02700354e-02, -1.74245105e-02,  4.60121796e-02,  8.77717050e-02,
         2.75587885e-02,  5.45849537e-02, -1.95268818e-02,  8.34163699e-03,
        -5.25343989e-02,  9.42755805e-02,  3.10933751e-02, -5.32311843e-02,
        -8.69914906e-02, -1.06020815e-01, -2.85367267e-02,  1.99596754e-02,
        -1.58125524e-01,  1.53246311e-02, -9.14944213e-03,  4.21959736e-02,
        -9.7

In [290]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:7]).transpose()

In [291]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()
# Calculate weights for eigenfaces
w = np.array([np.dot(proj_data,i) for i in np.array(A)])

In [296]:
# calculate eigenface of image in question
test_image = img_to_vector('../dataset/bush/George_W_Bush_0025.jpg')
mean_test_image = np.subtract(test_image, avg_face)

[26 28 32 ... 19 19 19]


In [297]:
print(test_image)
w_unknown = np.dot(proj_data, mean_test_image)
w_unknown

[26 28 32 ... 19 19 19]


array([ -3691824.49879999,  33289975.97215233, -13547063.14726994,
        17218165.63951493,  24684611.45491835,  15930356.66619541,
        -3819611.86436534])

In [298]:
diff  = w - w_unknown
print(diff.shape)
norms = np.linalg.norm(diff, axis=1)
print(norms.shape)
min(norms)
index = np.argmin(norms, axis=0)
index

(225, 7)
(225,)


60

In [299]:
vector_to_img(training_set[index]).show()